# Clusteranalyse mit DBSCAN

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns    
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from itertools import product
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [ ]:
## Aufruf von DBCV aus Dateiordner

from scipy.spatial.distance import euclidean
%run DBCV.py

In [ ]:
def Musterplot(ax, title, xlabel, ylabel, xticks, yticks):
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)

    ax.xaxis.set_tick_params(top='on', direction='in', width=1)
    ax.yaxis.set_tick_params(right='on', direction='in', width=1)
    
    ax.set_title(title,fontweight='bold',fontsize=14)
    
    ax.set_xlabel(xlabel,fontweight='bold',fontsize=12)
    ax.set_ylabel(ylabel,fontweight='bold',fontsize=12)
    
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)

In [ ]:
df = pd.read_csv("Datensatz_Clusteranalyse.csv", sep=";")

In [ ]:
df2 = df[['Ankunftszeit','Abfahrtszeit','Ladeanteil','Leistung_max (kW)','Verbrauch (kWh)']]
df1 = df[['Leistung_max (kW)','Verbrauch (kWh)']]

In [ ]:
## Skalierung der Merkmale

scaler = StandardScaler()
X = scaler.fit_transform(df2)

In [ ]:
## Bestimmung von Bereich für Epsilon

minPts = 10

nachbarn = NearestNeighbors(n_neighbors=minPts)
nachbarn_fit = nachbarn.fit(X)
distanz, index = nachbarn_fit.kneighbors(X)
distanz = np.sort(distanz, axis=0)
distanz = distanz[:,1]

fig, ax = plt.subplots(1,1,figsize=(7,5))
xlabel = "Datenpunkte mit minPts"
ylabel = "minimale Distanz"
title = "Bereich von Epsilon"
yticks = np.arange(0, 2.6, step=0.2)
xticks = np.arange(0, 1300, step=100)
plt.grid()
ax.set_axisbelow(True)
ax.plot(distanz)
Musterplot(ax, title, xlabel, ylabel, xticks, yticks)

plt.show()

fig.savefig("Bestimmung_Epsilon_DBSCAN")

In [ ]:
## Validierung von allen Kombination von Epsilon und minPts

eps_werte = np.round(np.arange(1.4,1.45,0.05),2)
min_punkte = np.arange(3,4) 
dbscan_params = list(product(eps_werte, min_punkte))

cluster = []
DBCV_score = []
epsi= []
minPts = []

for i in dbscan_params:
    dbscan = DBSCAN(eps=i[0], min_samples=i[1])
    Labels = dbscan.fit_predict(X)
    epsi.append(i[0])
    minPts.append(i[1])
    cluster.append(len(np.unique(Labels)))
    DBCV_score.append(DBCV(X, Labels, dist_function=euclidean))
    eps_min = list(zip(cluster, DBCV_score, epsi, minPts))

eps_min_df = pd.DataFrame(eps_min, columns=['cluster','DBCV_score','epsilon', 'minPts'])


In [ ]:
## Auswahl des Ergebnis mit dem höchsten DBCV 

erg = eps_min_df.iloc[eps_min_df['DBCV_score'].idxmax()]

In [ ]:
## Initalisierung von DBSCAN mit bester Kombination von minPts und Epsilon

eps = erg.epsilon
min_samples = int(erg.minPts)

dbscan = DBSCAN(eps = eps, min_samples=min_samples)
Labels = dbscan.fit_predict(X)

In [ ]:
## Zweidimensionales Beispiel (Ankunfts- und Abfahrtszeit)

# fig, ax = plt.subplots(1,1,figsize=(7,5))

# xlabel = "Ankunftszeit in h"
# ylabel = "Abfahrtszeit in h "
# title = "Clusterergebnis mit DBSCAN (Ankunfts- und Abfahrtszeit)"
# yticks = np.arange(5, 25, step=1)
# xticks = np.arange(5, 25, step=1)
# plt.grid()
# ax.set_axisbelow(True)
# sns.scatterplot(df2['Ankunftszeit'], df2['Abfahrtszeit'], hue = df2['Labels'],palette="deep")
# # ax.scatter(df['Ankunftszeit'],df['Abfahrtszeit'], c = df2['Labels'])
# # ax.scatter(kmeans.cluster_centers_[:,0] , kmeans.cluster_centers_[:,1] ,marker='x', s = 200, color = 'k')
# Musterplot(ax, title, xlabel, ylabel, xticks, yticks)

# plt.show()

# fig.savefig("Scatterplot_An_Ab_Beispiel_DBSCAN")

In [ ]:
## Zweidimensionales Beispiel (max Leistung und Gesamtverbrauch)

# fig, ax = plt.subplots(1,1,figsize=(7,5))

# xlabel = "max. Leistung in kW"
# ylabel = "Verbrauch in kWh"
# title = "Clusterergebnis mit DBSCAN (Leistung und Verbrauch)"
# yticks = np.arange(0, 60, step=5)
# xticks = np.arange(0, 22, step=1)
# plt.grid()
# ax.set_axisbelow(True)
# sns.scatterplot(df2['Leistung_max (kW)'], df2['Verbrauch (kWh)'], hue = df2['Labels'],palette="deep")
# # ax.scatter(df['Ankunftszeit'],df['Abfahrtszeit'], c = df2['Labels'])
# # ax.scatter(kmeans.cluster_centers_[:,0] , kmeans.cluster_centers_[:,1] ,marker='x', s = 200, color = 'k')
# Musterplot(ax, title, xlabel, ylabel, xticks, yticks)

# plt.show()

# fig.savefig("Scatterplot_L_V_Beispiel_DBSCAN")